In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
import pickle
import json
import cv2
import re
import keras

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
# make image dataloader using flow_from_dataframe
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# load data to extract labels
data_dir = 'mmhs150k/'
model_dir = 'models/'
tweet_dict = json.load(open(data_dir + 'MMHS150K_GT.json', 'r'))

In [22]:
# load pretrained LSTM and CNN for text and images
from keras.models import load_model

lstm = load_model(model_dir + 'lstm.h5')
cnn = load_model(model_dir + 'CNN.h5')

print(lstm.summary())
print(cnn.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           7565100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               150600    
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 151       
Total params: 7,738,501
Trainable params: 173,401
Non-trainable params: 7,565,100
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
___________

In [41]:
# create new models that are all but the last layer
from keras.models import Sequential, Model

text_net = Sequential()
for layer in lstm.layers[:-1]: text_net.add(layer)
print(text_net.summary())

img_net = Sequential()
for layer in cnn.layers[:-2]: img_net.add(layer)
print(img_net.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           7565100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               150600    
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
Total params: 7,738,350
Trainable params: 173,250
Non-trainable params: 7,565,100
_________________________________________________________________
None
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
___________

In [42]:
# build full FCM model using concatenation layer
from keras.layers.merge import concatenate
from keras.layers import Dense, Input

text_input = Input((text_net.layers[0].input_shape[-1],)) # get rid of None's in front
img_text_input = Input((text_net.layers[0].input_shape[-1],))
img_input = Input((img_net.layers[0].input_shape[1:])) # get rid of None in front

text_embed = text_net(text_input)
img_text_embed = text_net(img_text_input)
img_embed = img_net(img_input)

x = concatenate([text_embed, img_text_embed, img_embed])
x = Dense(2048 + 150 + 150, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

fcm_model = Model(inputs=[text_input, img_text_input, img_input], outputs=prediction)
print(fcm_model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
sequential_8 (Sequential)       (None, 150)          7738350     input_33[0][0]                   
                                                                 input_34[0][0]             